In [1]:
import IPython.core.display as di

# This line will hide code by default when the notebook is exported as HTML
di.display_html('<script>jQuery(function() {if (jQuery("body.notebook_app").length == 0) { jQuery(".input_area").toggle(); jQuery(".prompt").toggle();}});</script>', raw=True)

# This line will add a button to toggle visibility of code blocks, for use with the HTML export version
di.display_html('''<button onclick="jQuery('.input_area').toggle(); jQuery('.prompt').toggle();">Toggle code</button>''', raw=True)

Toggle code

# Udacity Data Analyst Nanodegree
# P3: Data Wrangling with MongoDB

Author: Luiz Gerosa

Date: Mar 7, 2017


## Introduction

As this [New York Times article](https://www.nytimes.com/2014/08/18/technology/for-big-data-scientists-hurdle-to-insights-is-janitor-work.html) points out, the less heralded part of doing data science is manually collecting and cleaning data so it can be easily explored and analyzed later. Or otherwise known as “data wrangling” or “data munging” in the data science community.

Though not as glamorous as building cool machine learning models, data wrangling is a task that data scientists can spend up to 50-80% of their time doing according to many practicing data analyst and data scientists.

The goal of this project is to choose any area of the world in https://www.openstreetmap.org and use data munging techniques, such as assessing the quality of the data for validity, accuracy, completeness, consistency and uniformity, to clean the OpenStreetMap data.

The following steps were executed in the wrangling process:
1. Audit the data and fix some inconsistencies found 
1. Reshape the data from the OSM file (XML format) into a JSON format.
1. Import the new file into MongoDB
1. Explore the data using MongoDB

This report outlines the results of the wrangling process, including:
* Problems encountered in the map
* Overview of the Data
* Other ideas about the dataset

### Map Area

The metropolitan area chosen for this project was Vila Velha, Brazil.

* [OpenStreetMap link](https://www.openstreetmap.org/relation/1825815)
* [OSM file](https://s3.amazonaws.com/mapzen.odes/ex_fFDuD5tRsSCfuQwG1w9MHwL27gfpB.osm.bz2)

### Tools used

* Python 3.6.0
* MongoDB 3.4.2

## Problems Encountered
The first step of the wrangling process is to audit the data to try to find and fix some inconsistencies.

### 1. Street Types

The first issue found in the dataset was over-abbreviated street types, e.g. "Rod." rather than "Rodovia", or typos, e.g. "Praca" rather than "Praça". The list below shows all occurences of streets that don't start with the expected types.

In [2]:
import xml.etree.cElementTree as ET
from collections import defaultdict
import re
import pprint

OSM_FILE = "dataset/vila_velha.osm"
street_type_re = re.compile(r'^\S+', re.IGNORECASE)

expected = ["Aeroporto","Alameda","Área","Avenida","Beco","Campo","Chácara","Colônia","Condomínio","Conjunto","Distrito","Escadaria","Esplanada","Estação","Estrada","Favela","Fazenda","Feira","Jardim","Ladeira","Lago","Lagoa","Largo","Loteamento","Morro","Núcleo","Parque","Passarela","Pátio","Praça","Quadra","Rampa","Recanto","Residencial","Rodovia","Rua","Setor","Sítio","Travessa","Trecho","Trevo","Vale","Vereda","Via","Viaduto","Viela","Vila"]

def audit_street_type(street_types, street_name):
    m = street_type_re.search(street_name)
    if m:
        street_type = m.group()
        if street_type not in expected:
            street_types[street_type].add(street_name)


def is_street_name(elem):
    return (elem.attrib['k'] == "addr:street")

def audit_street_types(osmfile):
    street_types = defaultdict(set)
    for event, elem in ET.iterparse(osmfile, events=("start",)):

        if elem.tag == "node" or elem.tag == "way":
            for tag in elem.iter("tag"):
                if is_street_name(tag):
                    audit_street_type(street_types, tag.attrib['v'])
    return street_types

st_types = audit_street_types(OSM_FILE)

pprint.pprint(st_types)

defaultdict(<class 'set'>,
            {'BAIRRO': {'BAIRRO PALMEIRAS'},
             'BR': {'BR 262', 'BR - 262'},
             'BR-101': {'BR-101'},
             'BR-262': {'BR-262'},
             'ESTRADA': {'ESTRADA 101 APOS POSTO Nossa Senhora do APARECIDA',
                         'ESTRADA CAMBOAPINA',
                         'ESTRADA PARA O ACQUAMANIA',
                         'ESTRADA PARA PALMEIRAS'},
             'FAZENDA': {'FAZENDA SAO DOMINGOS'},
             'Graciano': {'Graciano Neves'},
             'Mamoeiro': {'Mamoeiro'},
             'Praca': {'Praca Henrique Meyerfreund'},
             'RODOVIA': {'RODOVIA BR 262'},
             'Rod': {'Rod BR-101 - Contorno'},
             'Rod.': {'Rod. Es-080 - Governador José Sette'},
             'Servidão': {'Servidão Antônio Luiz Curvacho 1',
                          'Servidão Antônio Oliveira',
                          'Servidão Cantídio Moreira',
                          'Servidão Vitória Sant´Ana Ribeiro'},
       

Some street types were fixed using a dict mapping.

In [3]:
mapping = { "Rod": "Rodovia",
            "Rod.": "Rodovia",
            "RODOVIA": "Rodovia",
            "Praca": "Praça",
            "ESTRADA": "Estrada",
            "FAZENDA": "Fazenda"}

print('\nMapping:\n')
pprint.pprint(mapping)

def update_street_type(name, mapping):
    """Fixes the street type (i.e the first word) of a street using a dict mapping. 
    For example, if name == "Rod. BR-101" and mapping == {"Rod" : "Rodovia"}, this function returns "Rodovia BR-101"
    Args:
        name: the street name.
        mapping: the dict mapping used to fix the street types
        
    Returns:
        the street name replacing the street type by its corrected version.
    """
    m = street_type_re.search(name)
    if m:
        t = m.group()
        if t in mapping:
            return street_type_re.sub(mapping[t], name) 
    return name

print('\nResults after fixing the streets:\n')
# validate if the function is working correctly
for st_type, ways in st_types.items():
    for name in ways:
        fixed = update_street_type(name, mapping)
        if name != fixed:
            print("{} => {}".format(name, fixed))



Mapping:

{'ESTRADA': 'Estrada',
 'FAZENDA': 'Fazenda',
 'Praca': 'Praça',
 'RODOVIA': 'Rodovia',
 'Rod': 'Rodovia',
 'Rod.': 'Rodovia'}

Results after fixing the streets:

Rod. Es-080 - Governador José Sette => Rodovia Es-080 - Governador José Sette
ESTRADA 101 APOS POSTO Nossa Senhora do APARECIDA => Estrada 101 APOS POSTO Nossa Senhora do APARECIDA
ESTRADA PARA PALMEIRAS => Estrada PARA PALMEIRAS
ESTRADA PARA O ACQUAMANIA => Estrada PARA O ACQUAMANIA
ESTRADA CAMBOAPINA => Estrada CAMBOAPINA
RODOVIA BR 262 => Rodovia BR 262
FAZENDA SAO DOMINGOS => Fazenda SAO DOMINGOS
Praca Henrique Meyerfreund => Praça Henrique Meyerfreund
Rod BR-101 - Contorno => Rodovia BR-101 - Contorno


### 2. Zip Codes
Brazilian zip code is called CEP and uses the format 00000-000. Also, acording to [Correios](http://www.correios.com.br/) web site, the range of valid zip codes for Vila Velha is between 29100-001 to 29129-999.

In [4]:
zipcode_re = re.compile(r'^\d{5}\-\d{3}$')
digits_re = re.compile(r'\d+')

def extract_digits(str):
    digits = digits_re.findall(str)
    s = ""
    for d in digits:
        s += d
    
    return int(s)

def is_zipcode(elem):
    return (elem.attrib['k'] == "addr:postcode")

def audit_zipcode(invalid_zipcodes, zipcode):
    m = zipcode_re.search(zipcode)
    if m is None:
        invalid_zipcodes.add(zipcode)     

def audit_zipcode_range(zipcodes_out_of_range, zipcode):
    digits = extract_digits(zipcode)
    if digits < 29101001 or digits > 29129999:
        zipcodes_out_of_range.add(zipcode)

def audit_zipcodes(osmfile):
    invalid_zipcodes = set()
    zipcodes_out_of_range = set()
    for event, elem in ET.iterparse(osmfile, events=("start",)):
        if elem.tag == "node" or elem.tag == "way":
            for tag in elem.iter("tag"):
                if is_zipcode(tag):
                    audit_zipcode(invalid_zipcodes, tag.attrib['v'])
                    audit_zipcode_range(zipcodes_out_of_range, tag.attrib['v'])
    
    return invalid_zipcodes, zipcodes_out_of_range

invalid_zipcodes, zipcodes_out_of_range = audit_zipcodes(OSM_FILE)

print("Zip codes with invalid format:")
pprint.pprint(invalid_zipcodes)
print("\nNumber of zip codes out of range: {}".format(len(zipcodes_out_of_range)))

Zip codes with invalid format:
{'29.053-245',
 '29.100-200',
 '29010340',
 '29010906',
 '29017950',
 '29041265',
 '29047850',
 '29050705',
 '29100000',
 '29102195',
 '29102240',
 '3149-7327'}

Number of zip codes out of range: 258


The zip codes were standardized to the CEP format and checked if it's within the valid range. For example:

In [5]:
def update_zipcode(zipcode):
    digits = extract_digits(zipcode)
    if digits < 29101001 or digits > 29129999:
        return None
    else:
        fst_part = int(digits / 1000)
        snd_part = round((digits/1000 - fst_part) * 1000)
        return "{}-{}".format(fst_part, snd_part)

# validate if the function is working correctly
for invalid in invalid_zipcodes:
    print("{} => {}".format(invalid, update_zipcode(invalid)))

29102195 => 29102-195
3149-7327 => None
29.053-245 => None
29010340 => None
29100000 => None
29041265 => None
29017950 => None
29.100-200 => None
29050705 => None
29047850 => None
29102240 => 29102-240
29010906 => None


### Reshape Data to JSON
To be able to import the data into MongoDB, the data had to be reshaped from XML to JSON format. The following rules were applied:
* process only 2 types of top level tags: "node" and "way"
* all attributes of "node" and "way" should be turned into regular key/value pairs, except:
    - attributes in the CREATED array should be added under a key "created"
    - attributes for latitude and longitude should be added to a "pos" array,
      for use in geospacial indexing. Make sure the values inside "pos" array are floats
      and not strings. 
* if the second level tag "k" value contains problematic characters, it should be ignored
* if the second level tag "k" value starts with "addr:", it should be added to a dictionary "address"
* if the second level tag "k" value does not start with "addr:", but contains ":", you can
  process it in a way that you feel is best. For example, you might split it into a two-level
  dictionary like with "addr:", or otherwise convert the ":" to create a valid key.
* if there is a second ":" that separates the type/direction of a street,
  the tag should be ignored, for example:

```xml
<tag k="addr:housenumber" v="5158"/>
<tag k="addr:street" v="North Lincoln Avenue"/>
<tag k="addr:street:name" v="Lincoln"/>
<tag k="addr:street:prefix" v="North"/>
<tag k="addr:street:type" v="Avenue"/>
<tag k="amenity" v="pharmacy"/>
```

should be turned into:

```json
{...
"address": {
    "housenumber": 5158,
    "street": "North Lincoln Avenue"
}
"amenity": "pharmacy",
...
}
```

- for "way" specifically:

  <nd ref="305896090"/>
  <nd ref="1719825889"/>

should be turned into
```json
"node_refs": ["305896090", "1719825889"]
```

In [6]:
import xml.etree.cElementTree as ET
import pprint
import re
import io
import json

lower = re.compile(r'^([a-z]|_)*$')
lower_colon = re.compile(r'^([a-z]|_)*:([a-z]|_)*$')
problemchars = re.compile(r'[=\+/&<>;\'"\?%#$@\,\. \t\r\n]')

CREATED = [ "version", "changeset", "timestamp", "user", "uid"]
ADDRESS_TAGS = ["housenumber", "postcode", "street", "city", "place", "state"]

def shape_element(element):
    
    if element.tag == "node" or element.tag == "way" :
        node = {"type" : element.tag}

        # process the attributes
        for k, v in element.attrib.items():
            
            # created
            if k in CREATED:
                if "created" not in node:
                    node["created"] = {}
                node["created"][k] = v
            
            # others
            elif k != "lat" and k != "lon":
                node[k] = v

        # special attribute location
        if element.tag == "node":
            node["pos"] = [float(element.attrib["lat"]), float(element.attrib["lon"])]

        # process the 2nd level tags
        for tag in element.iter("tag"):
            
            k = tag.attrib["k"]
            v = tag.attrib["v"]

            if problemchars.search(k):
                continue

            if is_zipcode(tag):
                v = update_zipcode(v)
            elif is_street_name(tag):
                v = update_street_type(v, mapping)

            # tags that contains ":"
            split = k.split(":")
            if len(split) == 1:
                node[k] = v

            elif len(split) == 2:

                outer = split[0]
                inner = split[1]

                # address
                if outer == "addr":
                    if inner not in ADDRESS_TAGS:
                        continue
                    
                    outer = "address"
                    
                    if outer not in node:
                        node[outer] = {}
                        
                    node[outer][inner] = v
                else:
                    node[outer + "_" + inner] = v                
        
        # process node references
        for nd in element.iter("nd"):
            ref = nd.attrib["ref"]
            if "node_refs" not in node:
                node["node_refs"] = []
            node["node_refs"].append(ref)


        return node
    else:
        return None


def process_map(file_in, pretty = False):
    # You do not need to change this file
    file_out = "{0}.json".format(file_in)
    with io.open(file_out, "w", encoding="utf-8") as fo:
        for _, element in ET.iterparse(file_in):
            el = shape_element(element)
            if el:
                indent = None
                if pretty:
                    indent = 2

                fo.write(json.dumps(el, indent=indent, ensure_ascii=False)+"\n")

process_map(OSM_FILE, False)

## Import Data into MongoDB
[`mongoimport`](https://docs.mongodb.com/manual/reference/program/mongoimport/) was used to import the JSON file into a local MongoDB instance.

In [7]:
from pymongo import MongoClient

db_name = 'openstreetmap'

# Connect to MongoDB
client = MongoClient('localhost:27017')
db = client[db_name]

In [8]:
import subprocess

# Build mongoimport command
json_file = OSM_FILE + '.json'

collection = "vv"

mongoimport_cmd = 'mongoimport -h 127.0.0.1:27017 ' + \
                  '--db ' + db_name + \
                  ' --collection ' + collection + \
                  ' --file ' + json_file

# Before importing, drop collection if it is already running 
if collection in db.collection_names():
    print('Dropping collection: ' + collection)
    db[collection].drop()
    
# Execute the command
print('Executing: ' + mongoimport_cmd)
subprocess.call(mongoimport_cmd.split())

Dropping collection: vv
Executing: mongoimport -h 127.0.0.1:27017 --db openstreetmap --collection vv --file dataset/vila_velha.osm.json


0

## Data Overview
This section contains basic statistics about the dataset and the MongoDB queries used to gather them.

### File sizes

In [9]:
import os
print("The OSM file is {:4.2f} MB".format(os.path.getsize(OSM_FILE)/1.0e6)) # convert from bytes to megabytes
print("The JSON file is {:4.2f} MB".format(os.path.getsize(json_file)/1.0e6)) # convert from bytes to megabytes

The OSM file is 102.93 MB
The JSON file is 115.64 MB


### Number of documents

In [10]:
db.vv.find().count()

514356

### Number of nodes

In [11]:
db.vv.find({"type":"node"}).count()

451964

### Number of ways

In [12]:
db.vv.find({"type":"way"}).count()

62387

### Number of unique users

In [13]:
len(db.vv.distinct("created.user"))

163

### Top 10 contributing user

In [14]:
top = db.vv.aggregate([
    {"$group" : {"_id" : "$created.user", "count" : {"$sum" : 1}}},
    {"$sort" : {"count": -1}},
    {"$limit" : 10}])

pprint.pprint((list(top)))

[{'_id': 'trilhado', 'count': 251140},
 {'_id': 'LucFreitas', 'count': 130318},
 {'_id': 'Skippern', 'count': 92471},
 {'_id': 'BladeTC', 'count': 14454},
 {'_id': 'Thundercel', 'count': 12908},
 {'_id': 'Társis José', 'count': 2994},
 {'_id': 'isadorarf', 'count': 1230},
 {'_id': 'erickdeoliveiraleal', 'count': 1008},
 {'_id': 'abel801', 'count': 947},
 {'_id': 'wolvsky', 'count': 706}]


## Additional Ideas

### Top 5 most popular cuisines

In [15]:
top = db.vv.aggregate([
                   {"$match" : {"amenity":"restaurant"}},
                   {"$group" : {"_id" : "$cuisine", "count" : {"$sum" : 1}}},
                   {"$sort" : {"count" : -1}},
                   {"$limit": 5}])
pprint.pprint(list(top))

[{'_id': None, 'count': 46},
 {'_id': 'regional', 'count': 19},
 {'_id': 'pizza', 'count': 13},
 {'_id': 'steak_house', 'count': 7},
 {'_id': 'italian', 'count': 5}]


### Improving cuisine information

In [16]:
cuisines = list(db.vv.aggregate([
                   {"$match" : {"amenity":"restaurant"}},
                   {"$group" : {"_id" : "$cuisine", "count" : {"$sum" : 1}}},
                   {"$sort" : {"count" : -1}}]))

total = sum([c["count"] for c in cuisines])

print("\nRestaurants without cuisine defined: {:4.2f}%".format(cuisines[0]['count'] / total * 100))


Restaurants without cuisine defined: 41.82%


As can be observed, more than 40% of the restaurants don't have the cuisine field defined. This is an issue if this dataset would be used, for instance, by a restaurant recommendation app.

One way to improve the quality of the dataset is to try to infer the cuisine from the restaurant name. For example, if the restaurant has "Natural" on its name, we could infer it's a vegetarian restaurant.

In [17]:
query = {"amenity":"restaurant", "cuisine": {"$exists" : 0}, "name": {"$exists" : 1}}
projection = {"_id" : 0, "name" : 1}
restaurants_without_cuisine = [r["name"] for r in db.vv.find(query, projection)]

name_cuisine_mapping = {
    "Grelhados" : "steak_house",
    "Natural" : "vegetarian",
    "Outback" : "steak_house",
    "Vegetariano" : "vegetarian",
    "Caranguejo" : "seafood",
    "Burger" : "burger",
    "Steakhouse" : "steak_house",
    "Ondas" : "seafood",
    "Mar" : "seafood",
    "Terra": "vegetarian"}
print('Mapping:\n')
pprint.pprint(name_cuisine_mapping)


print('\nCuisines inferred from the restaurant\'s name:\n')
for name in restaurants_without_cuisine:
    for k, v in name_cuisine_mapping.items():
        if k in name:
            print("{} => {}".format(name, v))

Mapping:

{'Burger': 'burger',
 'Caranguejo': 'seafood',
 'Grelhados': 'steak_house',
 'Mar': 'seafood',
 'Natural': 'vegetarian',
 'Ondas': 'seafood',
 'Outback': 'steak_house',
 'Steakhouse': 'steak_house',
 'Terra': 'vegetarian',
 'Vegetariano': 'vegetarian'}

Cuisines inferred from the restaurant's name:

Grelhados e Massas Restaurante => steak_house
Sabor Natural => vegetarian
Restaurante Sol & Mar => seafood
Outback => steak_house
Restaurante Vegetariano => vegetarian
Terra do Sabor Restaurante e Self-Service => vegetarian
Belas Ondas Restaurante => seafood
Ilha do Caranguejo => seafood
Bifão Steakhouse => steak_house
Brazillian Burger => burger
Restaurante Mar e Terra => seafood
Restaurante Mar e Terra => vegetarian


This approach has some issues that have to be addressed. 

For instance, "Mar" (sea in Portuguese) was mapped to "seafood" and "Terra" (soil in Portuguese) was mapped to "vegetarian". The restaurant "Restaurante Mar e Terra" has both words on its name and the classification could be wrong. In this case, we could define a prioritization in the mapping to classify to seafood rather than vegetarian, as vegetarian restaurants are not supposed to cook any kind of animal.

Another problem could be the wrong classifications. For example, "STK South Beach" is a steak house in Miami that would be wrong classified as seafood because of the word "Beach". In this case we could use other sources like Google Maps or Facebook to enhance the dataset.